# Flights Data Analysis

<div style="text-align: center;">
    <img src="https://www.airport-technology.com/wp-content/uploads/sites/14/2022/01/shutterstock_758602234-min-scaled-e1641297696653.jpg" alt="Airport Image" style="width:800px;">
</div>

In [ ]:
import sqlite3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import warnings
warnings.filterwarnings('ignore')
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] =  (22,5)
plt.rcParams['font.size'] = 14
plt.rcParams['axes.spines.top'] = False
plt.rcParams['axes.spines.right'] = False

In [ ]:
df =  pd.read_csv('/kaggle/input/goibibo-flight-data/goibibo_flights_data.csv')
df = df.drop(columns= ['Unnamed: 11', 'Unnamed: 12'])
df.rename(columns={'flight date': 'flight_date', 'from': 'start', 'to': 'end'}, inplace=True)
df['flight_date'] = pd.to_datetime(df.flight_date, dayfirst=True)
df['price'] = df.price.str.replace(',', "")
df['price'] = pd.to_numeric(df.price)
df['duration'] = pd.to_numeric(df['duration'].str.replace("h ","").str.replace("m", ""))/100
df['Flight_Day'] = df['flight_date'].dt.strftime('%a')
df.head()

In [ ]:
df.dtypes

In [ ]:
df.info()

No Null Values in the dataset

In [ ]:
df.describe()

In [ ]:
n_rows, n_cols = df.shape
print(f"There are {n_rows:,} rows and {n_cols} columns in the dataset.")

In [ ]:
conn = sqlite3.connect('flights.db')
df.to_sql('flight_data', conn, if_exists='replace', index=False)

In [ ]:
n_airlines = len(df.airline.unique().tolist())
n_flights = len(df.flight_num.unique().tolist())
print(f"There are {n_flights:,} flights of {n_airlines} airlines.")

Row Granularity is flight date meaning 1 row is 1 flight.

## Airline Analysis

Which airline operates the most flights?

In [ ]:
query = '''
select airline, 
        round(count(*)*100.0/(select count(*) from flight_data),2) as Flights_Percent 
from flight_data 
group by 1 
order by 2 desc
'''
flights_pct = pd.read_sql(query, conn)
flights_pct

- **Observation**: Vistara operates the highest proportion of flights, with 42.58% of the total, indicating it is the most frequently flying airline in this dataset. In comparison, Air India, the second most frequent, accounts for 26.94% of the flights, while other airlines operate at significantly lower frequencies.

- **So What?**
   - With the highest flight availability, Vistara may be the default choice for many travelers, particularly on routes where it operates frequently. This could give Vistara an advantage in attracting more passengers, especially those seeking convenience in scheduling.
   - Airlines like StarAir and Trujet, with very low percentages (0.02% and 0.01%, respectively), may have limited market presence, likely serving niche routes or offering fewer services overall.

- **Action:**

    - For Vistara: Consider maintaining or increasing frequency on high-demand routes to capitalize on market presence and explore loyalty programs to enhance customer retention. 
    - For Other Airlines: Air India and Indigo could explore popular routes Vistara serves, assessing the potential for increased flight frequencies or added routes. For airlines with a smaller market share, focusing on niche routes, customer-specific needs, or strategic partnerships could improve visibility and market penetration.

Which airline has the highest/lowest average price?

In [ ]:
query = '''
select airline, 
    max(price) as MaxPrice,
    min(price) as MinPrice,
    max(price) - min(price) as Range
from flight_data 
group by 1 
'''
prices = pd.read_sql(query, conn)
prices

In [ ]:
df.groupby('airline')['price'].agg(mean='mean', std='std', count='count')\
                              .apply(lambda x: round(x,2))\
                              .sort_values('count', ascending=False)

Observation:

1. Price Disparity: Vistara has the highest mean price at ₹30,831.96, significantly more than other airlines, with a standard deviation of ₹26,038.30, indicating a wide range of prices. In contrast, Trujet has the lowest mean price at ₹3,277.10, with a standard deviation of ₹308.17, suggesting more consistent pricing.
2. Passenger Volume: Vistara has the highest count of tickets sold (127,859), indicating strong demand despite its higher prices, while StarAir and Trujet show much lower counts (61 and 41 respectively).
3. Market Positioning: Airlines like Air India and Indigo have a mean price of ₹23,837.56 and ₹5,377.49, respectively, suggesting a balance between affordability and demand.

So What?

- Market Segmentation: Vistara’s high mean price coupled with a substantial number of sold tickets indicates that it is effectively targeting premium customers. This could imply a successful branding strategy focused on quality and service.
- Revenue Potential: The significant standard deviation in Vistara’s prices suggests there might be opportunities for promotional pricing or varied fare structures to attract different customer segments without diluting the brand.
- Competitive Strategy: Airlines like Indigo and AirAsia, with lower average prices, may be capitalizing on budget-conscious travelers. Their relatively lower standard deviations indicate more stable pricing, which could appeal to a different customer segment seeking predictability in costs.

Action:

- For Vistara: Continue leveraging its premium positioning while exploring targeted promotions to attract more price-sensitive travelers, which could boost overall sales without compromising brand integrity.
- For Budget Airlines: Indigo and AirAsia should maintain their competitive pricing strategies while evaluating the feasibility of introducing additional services or enhancements to attract more business travelers and increase revenue per passenger.
- Overall: All airlines should conduct customer feedback sessions to understand preferences related to pricing and services, allowing them to refine their offerings and marketing strategies effectively.

What is the average duration of flights for each airline?

In [ ]:
query = '''
select airline, 
    round(avg(duration),2) as avg_duration
from flight_data 
group by 1 
'''
duration = pd.read_sql(query, conn)
duration

Observation:

1. Longest Average Duration: Air India has the longest average flight duration at 15.32 hours, followed closely by Vistara at 13.16 hours. This suggests that these airlines may primarily operate routes.
2. Shortest Average Duration: StarAir shows the shortest average flight duration at just 2.41 hours, indicating a focus on shorter regional flights.
3. Mid-Range Durations: Airlines like AirAsia (8.76 hours) and GO FIRST (8.57 hours) fall in the mid-range, suggesting a mix of short and medium-haul flights.

So What?

- Route Strategy: The longer average durations for Air India and Vistara imply that they may cater to business or premium travelers who prioritize comfort on long-haul flights. Understanding this could help in tailoring services and pricing strategies.
- Market Differentiation: Airlines like StarAir focusing on shorter durations may appeal to budget-conscious travelers seeking quick trips. This positioning can be advantageous in a competitive market where time efficiency is crucial.

Action:

- Service Improvement for Long-Haul Flights: Air India and Vistara should ensure that their services (in-flight amenities, food, etc.) meet the expectations of travelers on longer flights. Customer feedback can be used to enhance these offerings.
- Promote Short-Haul Options: StarAir should leverage its short flight durations in marketing campaigns targeting travelers looking for quick and affordable travel options.
- Analyze Duration Trends: All airlines should continuously monitor average flight durations and adapt their schedules and routes to optimize operational efficiency and meet passenger demands.

In [ ]:
query = '''
select airline, 
    round(sum(case when class = 'business' then 1 else 0 end)*100.0/count(*),2) as business,
    round(sum(case when class = 'economy' then 1 else 0 end)*100.0/count(*),2) as economy
from flight_data 
group by 1
order by 2 desc, 3 desc, 1 asc
'''
class_pct = pd.read_sql(query, conn)
class_pct

- **Observation**: Vistara offers a balanced mix of flight classes, with 47.39% of its flights in business class and 52.61% in economy class. In contrast, Air India has 40.67% business and 59.33% economy. Other airlines, including AirAsia, GO FIRST, Indigo, SpiceJet, StarAir, and Trujet, exclusively offer economy class flights (100%).

- **So What?**: The diverse class offering from Vistara and Air India indicates a strategy that caters to different market segments. Vistara’s relatively high percentage of business class flights (47.39%) suggests it targets both business travelers and leisure customers, which may enhance revenue opportunities. In contrast, the other airlines’ focus on economy-only flights suggests a potential missed opportunity to attract higher-paying customers who prefer premium services.

- **Action**: Vistara should leverage its dual-class offering in marketing campaigns to attract business travelers and those seeking premium experiences. Air India could similarly emphasize its business class services to differentiate itself from low-cost carriers. Airlines exclusively offering economy class should consider diversifying their offerings to include business class options to tap into a potentially lucrative market segment. Additionally, Vistara can analyze customer feedback and preferences to further enhance its business class experience, ensuring it meets the needs of premium travelers.

Which airlines gives cheaper seats?

In [ ]:
query = '''
WITH Prices AS (
    SELECT 
        airline, 
        class, 
        ROUND(SUM(price),2) AS Price
    FROM flight_data
    GROUP BY 1,2
)

SELECT 
    a.airline, 
    a.class, 
    a.Price, 
    COUNT(CASE WHEN f.class = 'economy' THEN 1 END) AS Eco_Seats
FROM Prices a JOIN flight_data f ON a.airline = f.airline AND a.class = f.class
WHERE a.class = 'economy' 
GROUP BY 1,2,3
'''
price_seats = pd.read_sql(query, conn)
price_seats['Price_per_seat'] = round(price_seats['Price'] / price_seats['Eco_Seats'],2)
price_seats = price_seats.sort_values(by='Price_per_seat', ascending=False).reset_index(drop=True)
price_seats

**Observations:**

1. Lowest Average Price: Trujet has the lowest average price at ₹3,277, but it offers only 41 available seats, indicating a strategy targeting price-sensitive customers with limited capacity.
2. Highest Average Price: Vistara leads with the highest average price of ₹7,885 and a substantial seat availability of 67,270, reflecting a premium market positioning.
3. Price Range: Average prices range from ₹3,277 (Trujet) to ₹7,885 (Vistara), showcasing a diverse market catering to different customer segments.

**So What?**

1. Market Positioning: Trujet appears to be focused on budget travelers, while Vistara aims for a premium experience. This indicates contrasting business strategies between the airlines.
2. Capacity vs. Pricing: Airlines like Indigo and Air India have a balanced approach with competitive pricing and significant seat availability, positioning themselves well to capture larger market shares while ensuring profitability.

**What to Do:**

1. For Low-Cost Airlines: Trujet should explore increasing its seat capacity to maximize the potential of its low pricing strategy. Enhancing marketing efforts could improve brand visibility and competitiveness.
2. For Premium Airlines: Vistara should leverage its premium pricing to enhance customer services, ensuring that the perceived value justifies the higher fares.
3. For All Airlines: Airlines should analyze customer preferences related to pricing and capacity, adjusting their strategies to improve market share and overall performance.

## Airport Analysis

What are the top 5 busiest airports by departures/arrivals?

In [ ]:
query = '''
select 
    start, 
    count(*) as departures
from flight_data 
group by 1
order by 2 desc
limit 5
'''
busiest = pd.read_sql(query, conn)
busiest

Observation:

- Highest Departures: Delhi has the highest number of departures at 61,345, followed by Mumbai with 60,903. Bangalore, Kolkata, and Hyderabad show significantly lower figures, indicating a clear hub of activity centered in Delhi.

So What?

- Central Hub: The high number of departures from Delhi suggests it serves as a major connecting point for various destinations, indicating its importance in the airline network. This concentration can mean that travelers frequently transit through Delhi, making it a crucial part of the overall travel ecosystem.
- Service Quality: Given Delhi’s position as a primary departure point, ensuring high-quality services and operational efficiency is critical. Delays or service failures here could significantly impact a larger number of travelers.

Action:

- Enhance Services: Airlines operating from Delhi should prioritize maintaining and improving service levels, such as timely departures and customer support, to ensure a positive travel experience.
- Capacity Planning: As the hub grows, airlines may need to assess and enhance capacity to handle peak travel times effectively, ensuring they can accommodate the demand without compromising service quality.
- Infrastructure Coordination: Collaborate with airport authorities to ensure that the necessary infrastructure supports the high volume of departures, facilitating smoother operations and customer experiences.

In [ ]:
query = '''
select 
    end, 
    count(*) as arrivals
from flight_data 
group by 1
order by 2 desc
limit 5
'''
busiest = pd.read_sql(query, conn)
busiest

Observation:

- Most Arrivals: Mumbai has the highest number of arrivals at 59,109, closely followed by Delhi with 57,361. Bangalore, Kolkata, and Hyderabad show lower figures, indicating that Mumbai is a significant destination for travelers.

So What?

- Key Destination: The high number of arrivals in Mumbai highlights its status as a major destination city, which could be attributed to its economic, cultural, and entertainment significance. This makes it vital for airlines to optimize their services to and from this hub.
- Competition with Delhi: The close competition between Mumbai and Delhi in terms of arrivals suggests that both cities are critical nodes in the airline network, each serving distinct traveler needs. Airlines should consider market positioning and customer preferences in these regions.

Action:

- Optimize Services to Mumbai: Airlines should focus on enhancing services, including flight frequency and capacity, to meet demand in Mumbai. This could involve marketing campaigns to promote travel to and from this key destination.
- Strategic Partnerships: Form partnerships with local businesses and attractions to offer value-added services to travelers arriving in Mumbai, thereby enhancing customer satisfaction and loyalty.
- Capacity and Infrastructure Planning: Similar to departures from Delhi, airlines and airport authorities in Mumbai need to ensure that infrastructure and capacity can support the high volume of arrivals, particularly during peak travel seasons.

## Class Analysis

What is the average price for economy/business/first class?

In [ ]:
query = '''
select 
    class, 
    round(avg(price),2) as AvgPrice
from flight_data 
group by 1
order by 2 desc
'''
class_price = pd.read_sql(query, conn)
class_price

Observation:

1. Average Price for Business Class: Business class has a significantly higher average price at ₹53,328.19, indicating a premium market segment.
2. Average Price for Economy Class: Economy class has a much lower average price of ₹6,636.92, making it accessible to a broader audience.

So What?

- Market Segmentation: The stark difference in average prices suggests a clear segmentation in the market. Airlines can target different customer segments based on their price sensitivity, with business travelers willing to pay a premium for added comfort and services.
- Revenue Opportunities: The high average price for business class indicates substantial revenue potential. Airlines can focus on enhancing business class offerings to attract more corporate clients, especially in competitive markets.

Action:

- Enhance Business Class Offerings: Airlines should consider improving in-flight services, amenities, and overall experiences in business class to justify the premium pricing and attract more passengers.
- Promote Economy Class Packages: Marketing strategies for economy class can emphasize value for money, targeting price-sensitive travelers while ensuring that quality is maintained.
- Monitor Pricing Strategies: Airlines should regularly analyze their pricing structures for both classes to ensure competitiveness and profitability, adjusting based on market demand and customer preferences.

Let's find out why business class prices are so inflated.

In [ ]:
query = '''
select 
    airline, 
    round(avg(case when class='economy' then price else 0 end),2) as Eco_AvgPrice,
    round(avg(case when class='business' then price else 0 end),2) as Biz_AvgPrice
from flight_data 
group by 1
'''
airline_class = pd.read_sql(query, conn)
airline_class

Observation:

1. Vistara and Air India Business Class Prices: Vistara and Air India are the only airlines offering business class options, with average prices at ₹26,683.44 and ₹19,454.77, respectively. This exclusivity in offering business class is a key driver behind the high average price for this class overall.
2. Economy Class Prices Across Airlines: Economy fares vary less drastically across airlines, with Trujet having the lowest average price at ₹3,277.10 and SpiceJet at ₹6,241.05. Vistara’s economy fare is also relatively affordable at ₹4,148.52 despite its higher business class prices.

So What?

- Market Positioning in Business Class: Vistara’s premium pricing strategy for business class helps position it as a luxury carrier. This distinct pricing also signals a strategic focus on business travelers seeking premium services, which Air India also taps into, though at a lower price point.
- Economy Class Competitiveness: The close range of economy prices suggests a competitive market. Airlines with lower economy fares, like Trujet and AirAsia, may be leveraging low-cost strategies to attract budget travelers, while Vistara’s competitive economy pricing keeps it relevant for both segments.

Action:

- Enhance Business Class Differentiation: Vistara and Air India could continue to strengthen business class services to justify the premium further. This could include enhancing onboard services, amenities, and loyalty rewards.
- Targeted Marketing for Economy Travelers: Low-cost carriers like Trujet and AirAsia should focus on promoting value for money in their economy offerings. Vistara could also capitalize on its mid-range economy pricing to attract a broader segment, potentially gaining loyalty from economy passengers who may eventually consider upgrading to business.
- Pricing Strategy Reviews: Both Vistara and Air India could periodically evaluate the demand for business class to ensure their pricing remains aligned with customer expectations and market trends.

What is the flight count for economy/business/first class weekday wise?

In [ ]:
query = '''
SELECT  
    CASE CAST(strftime('%w', flight_date) AS integer)
        WHEN 0 THEN 'Sunday'
        WHEN 1 THEN 'Monday'
        WHEN 2 THEN 'Tuesday'
        WHEN 3 THEN 'Wednesday'
        WHEN 4 THEN 'Thursday'
        WHEN 5 THEN 'Friday'
        ELSE 'Saturday' 
    END AS weekday,
    SUM(CASE WHEN class = 'business' THEN 1 ELSE 0 END) AS Biz_Flights,
    SUM(CASE WHEN class = 'economy' THEN 1 ELSE 0 END) AS Eco_Flights    
FROM flight_data 
GROUP BY weekday
ORDER BY CASE weekday 
            WHEN 'Monday' THEN 1
            WHEN 'Tuesday' THEN 2
            WHEN 'Wednesday' THEN 3
            WHEN 'Thursday' THEN 4
            WHEN 'Friday' THEN 5
            WHEN 'Saturday' THEN 6
            ELSE 7 
         END;
'''
class_weekday= pd.read_sql(query, conn)
class_weekday.set_index('weekday').plot(kind='line', marker='o')
yticks = plt.gca().get_yticks()
ylabels = [f"{float(round(i / 1000,2))}k" for i in yticks]
plt.gca().set_yticklabels(ylabels)
plt.title('Weekday Wise Flight Count for each Class Seat')
plt.ylabel('Flight count')
plt.legend(loc='upper left')
plt.xticks(rotation=0)
plt.show()

Observation:

1. Highest Demand Days: Thursday has the highest number of business and economy flights, with business flights at 13,845 and economy flights at 30,228. This trend suggests peak mid-week demand.
2. Weekend Stability: Friday to Sunday sees a similar volume in both business and economy flights, with slight variances, indicating stable weekend demand.
3. Lowest Demand Day: Monday has the lowest number of both business and economy flights, possibly indicating reduced demand at the start of the week.

So What?

- Weekday Peak Demand: The high volume on Thursday suggests increased travel for work, events, or mid-week business needs, and airlines could explore higher ticket prices or special services on these days.
- Weekend Stability: With Friday to Sunday being consistently busy, airlines may consider additional services for leisure travelers or prioritize consistent service quality to capture repeat weekend customers.

Actions:

1. For High-Demand Days: Allocate additional resources or flights on Thursdays to meet peak demand, with the potential for premium pricing for business travelers.
2. Weekend Focus: Invest in marketing campaigns for weekend flights and loyalty programs targeting frequent weekend travelers to maximize occupancy and brand loyalty.
3. Monday Strategy: For lower-demand Mondays, explore discounted pricing or flexible ticketing to boost flight occupancy and attract cost-sensitive passengers.

## Time Analysis

In [ ]:
flight_date = df.groupby('flight_date')['airline'].count()
flight_date.plot(marker='o', color='mediumpurple')
yticks = plt.gca().get_yticks()
ylabels = [f"{float(round(i / 1000,2))}k" for i in yticks]
plt.gca().set_yticklabels(ylabels)
plt.title('Date Wise Flight Count, Overall')
plt.ylabel('flight count')
plt.legend(loc='upper right')
plt.show()

In [ ]:
month_flights = flight_date.groupby(flight_date.index.month).sum()
month_flights.plot(kind='bar')
yticks = plt.gca().get_yticks()
ylabels = [f"{round(i / 1000)}k" for i in yticks]
plt.gca().set_yticklabels(ylabels)
plt.title('Month Wise Flight Count, Overall')
plt.ylabel('flight count')
plt.xlabel('Month')
plt.legend(loc='upper right')
plt.xticks(rotation=0)
plt.show()

Observation:

- Seasonal Demand Peaks: July has the highest number of flights (196,734), indicating peak travel demand, followed by August (82,845) and June with the lowest (20,682).

So What?

- Summer Travel Demand: The spike in July and August suggests increased travel during these months, likely due to summer vacations or holiday periods.
- June Underperformance: The lower flight count in June could imply a ramp-up to peak travel season, or it may highlight untapped potential for promotional efforts to increase demand early.

Actions:

- Capacity Planning for July and August: Ensure sufficient capacity, staffing, and operational readiness to handle peak travel volumes efficiently.
- June Demand Stimulation: Consider early summer promotions or partnerships to encourage higher travel volumes in June, smoothing out monthly demand and maximizing revenue potential.

In [ ]:
pivot = df.pivot_table(index='flight_date', columns='airline', aggfunc='size', fill_value=0)\
          .sort_index(ascending=True)
pivot.plot()
yticks = plt.gca().get_yticks()
ylabels = [f"{float(round(i / 1000,2))}k" if abs(i) > 500 else str(int(i)) for i in yticks]
plt.gca().set_yticklabels(ylabels)
plt.title('Date Wise Flight Count for each Airline')
plt.ylabel('flight count')
plt.legend(loc='upper right')
plt.show()

- **Observation:** Air India and Vistara both started strong in terms of flight counts. However, within a week, Air India’s flight count stagnated at around 1500 flights a day while Vistara doubled its count and reached ~2700 flights a day before stagnating there. Indigo exhibited a similar initial growth but stagnated around 500 flights a day.

- **So What?** : Vistara’s ability to aggressively increase its flight count indicates a proactive approach to capturing market demand, suggesting effective strategies to attract more passengers. In contrast, Air India’s stagnation may imply complacency or operational challenges in expanding its services. Indigo’s plateau suggests it may have reached a capacity limit or is maintaining a steady, less aggressive growth strategy compared to Vistara.

- **Action:** Vistara should capitalize on its growth momentum by further promoting its services and potentially exploring new routes to maximize revenue. Air India might need to analyze its operational strategies and market positioning to identify barriers to growth and adapt accordingly. Indigo should evaluate its current service offerings to explore opportunities for increased flights or enhanced customer engagement to drive growth.

In [ ]:
price_pivot = df.pivot_table(index='flight_date',columns='airline',
                                 values='price', aggfunc='mean', fill_value=0)\
                .sort_index(ascending=True)
price_pivot.plot()
yticks = plt.gca().get_yticks()
ylabels = [f"{round(i / 1000)}k" if i != 0 else "0" for i in yticks]
plt.gca().set_yticklabels(ylabels)
plt.title('Date Wise Flight Price for each Airline')
plt.ylabel('flight price')
plt.legend(loc='upper right')
plt.show()

- **Observation**: 
    - Overall the flight prices have been coming down but Vistara consistently commands the highest average ticket prices throughout the analyzed period, with prices peaking around June 26 and remaining relatively high. 
    - Indigo maintains competitive pricing but shows slight fluctuations, while Air India exhibits a downward trend in average prices towards the end of July. 
    - Meanwhile, airlines like StarAir and Trujet frequently report prices of zero, indicating possible operational inactivity or unavailability of flights.


- **So What?**: Vistara’s high average prices suggest strong brand positioning and perceived value among customers, allowing it to maintain profitability. Indigo’s price fluctuations may indicate attempts to adjust to market demand or competitive pressures. Air India’s declining prices could signal challenges in maintaining market share or efforts to fill seats, possibly at the expense of profit margins. The zero prices for StarAir and Trujet point to significant operational issues, potentially impacting their ability to attract customers and generate revenue.

- **Action**: Vistara should capitalize on its strong price positioning by enhancing its service offerings and marketing strategies to reinforce its premium brand image. Indigo should analyze pricing strategies to balance competitiveness and profitability, possibly considering promotional offers during off-peak times. Air India needs to evaluate its pricing structure and operational strategies to address any underlying issues affecting price and demand. StarAir and Trujet must urgently assess their operations to resume flights and establish a pricing strategy that can effectively attract customers back into the market.